# Mount Google Drive to access files

---
# Imports & Config  

## Cloud Setup

In [0]:
#!gsutil mb gs://holoselecta_objectdetection2

Creating gs://holoselecta_objectdetection2/...


In [0]:
#!curl -H "Authorization: Bearer $(gcloud auth print-access-token)" https://ml.googleapis.com/v1/projects/holoselecta:getConfig

{
  "serviceAccount": "service-454653118122@cloud-ml.google.com.iam.gserviceaccount.com",
  "serviceAccountProject": "474819640395",
  "config": {
    "tpuServiceAccount": "service-474819640395@cloud-tpu.iam.gserviceaccount.com"
  }
}


In [0]:
#!gcloud projects add-iam-policy-binding holoselecta  --member serviceAccount:service-474819640395@cloud-tpu.iam.gserviceaccount.com --role roles/ml.serviceAgent

Updated IAM policy for project [holoselecta].
bindings:
- members:
  - serviceAccount:service-454653118122@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:454653118122-compute@developer.gserviceaccount.com
  - serviceAccount:454653118122@cloudservices.gserviceaccount.com
  role: roles/editor
- members:
  - serviceAccount:service-454653118122@cloud-ml.google.com.iam.gserviceaccount.com
  - serviceAccount:service-474819640395@cloud-tpu.iam.gserviceaccount.com
  role: roles/ml.serviceAgent
- members:
  - user:hyve.tobiasgrundmann@gmail.com
  role: roles/owner
etag: BwWKztbqzAI=
version: 1


## Config

In [0]:
import os

base_dir = "/content/gdrive/My Drive/Masterarbeit"
object_detection_dir = os.path.join(base_dir, "ObjectDetection")
image_classification = os.path.join(base_dir, "ImageClassification")

classes_dict_file = object_detection_dir + "/classes.pkl"
objects_dict_file = object_detection_dir + "/objects.pkl"
images_dict_file = object_detection_dir + "/images.pkl"

## Mount GDrive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from google.colab import auth
auth.authenticate_user()

In [4]:
!gcloud config set project objectdetection-229309

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud alpha survey



In [15]:
!gsutil -m cp -r gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/* gdrive/My\ Drive/Masterarbeit/ObjectDetection/Models/mobilenet_320_1000/

Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/model.ckpt-0.index...
Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/model.ckpt-0.meta...
Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/model.ckpt-5000.index...
Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/graph.pbtxt...
Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/model.ckpt-0.data-00000-of-00001...
Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/events.out.tfevents.1560617991.969541d12ec2...
Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/checkpoint...
Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/model.ckpt-5000.data-00000-of-00001...
Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/model.ckpt-5000.meta...
Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/pipeline.config...
Copying gs://holoselecta_objectdetection/mobilenetv1_4/320/1000/eval_validation_data/events.out.tf

## Imports

In [0]:
!git clone https://github.com/tensorflow/models.git /content/tf
!cd /content/tf/research; protoc object_detection/protos/*.proto --python_out=.
!cd /content/tf/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python /content/tf/research/object_detection/builders/model_builder_test.py

Cloning into '/content/tf'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 27331 (delta 0), reused 1 (delta 0), pack-reused 27325
Receiving objects: 100% (27331/27331), 509.03 MiB | 35.04 MiB/s, done.
Resolving deltas: 100% (16671/16671), done.
Checking out files: 100% (2985/2985), done.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.103s

OK (skipped=1)


In [0]:
!cd /content/tf/research; bash object_detection/dataset_tools/create_pycocotools_package.sh /tmp/pycocotools
!cd /content/tf/research; python setup.py sdist
!cd /content/tf/research; (cd slim && python setup.py sdist)

Cloning into 'cocoapi'...
remote: Enumerating objects: 953, done.
remote: Total 953 (delta 0), reused 0 (delta 0), pack-reused 953
Receiving objects: 100% (953/953), 11.70 MiB | 26.44 MiB/s, done.
Resolving deltas: 100% (565/565), done.
running sdist
running egg_info
creating object_detection.egg-info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
writing manifest file 'object_detection.egg-info/SOURCES.txt'
writing manifest file 'object_detection.egg-info/SOURCES.txt'
running check


creating object_detection-0.1
creating object_detection-0.1/a3c_blogpost
creating object_detection-0.1/adv_imagenet_models
creating object_detection-0.1/adversarial_crypto
creating object_detection-0.1/adversarial_logit_pairing
creating object_detection-0.1/adversarial_logit_pairing/datasets
creating object_

In [0]:
import os
import sys

# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.
#import matplotlib.pyplot as plt
#import tempfile
#from six.moves.urllib.request import urlopen
#from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import cv2

# For measuring the inference time.
from datetime import datetime

#for xml parsing
from bs4 import BeautifulSoup

#...
sys.path.append('tf/research')
sys.path.append('tf/research/slim')
from object_detection.utils import dataset_util, config_util
from object_detection import model_main

W0608 10:28:19.911887 139737064908672 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
class FLAGS_OD:
  image_dir = os.path.join(object_detection_dir, "Images")
  
  #tf records directory
  tf_records_dir = os.path.join(object_detection_dir, "TFRecords")
  #label_map directory
  label_map_dir = os.path.join(object_detection_dir, "LabelMaps")
  
  #class level to be considered
  class_level = [0,1]
  #how often the class of an object needs to occure to be considered
  min_class_occurence = 20
  
  #dataset split
  testing_percentage = 0
  validation_percentage = 10
  
  #pipeline config
  pipeline_config_dir = os.path.join(object_detection_dir, "ConfigFiles")
  
  #model checkpoints
  model_checkpoint_dir = os.path.join(object_detection_dir, "ModelCheckpoints")
  model_dir = os.path.join(object_detection_dir,"Models")
  
  #cloud configs
  cloud_config_dir = os.path.join(object_detection_dir, "CloudConfigFiles")

## Copy Data


In [0]:
!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/ConfigFiles/faster_rcnn_inception_resnet_v2_atrous_holoselecta.config pipeline.config

# Run

In [0]:
import time
from subprocess import call

res = 640
for it in [20, 10]:
  print(res)
  ret1 = call('gsutil cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/TFRecords/2019_5_28_9_24_26_test_{}_{}.record gs://holoselecta_objectdetection2/holoselecta/data/{}/{}/val.record'.format(res,it,res,it), shell=True)
  ret2 = call('gsutil cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/TFRecords/2019_5_28_9_24_26_train_{}_{}.record gs://holoselecta_objectdetection2/holoselecta/data/{}/{}/train.record'.format(res,it,res,it), shell=True)
  ret3 = call('gsutil cp pipeline_{}_{}.config gs://holoselecta_objectdetection2/inception/{}/{}/pipeline.config'.format(res,it,res,it), shell=True)
  
  time.sleep(20)
  
  cl_str = ('cd /content/tf/research; ' +
       'gcloud ml-engine jobs submit training object_detection_`date +%m_%d_%Y_%H_%M_%S` ' +
       '--python-version 3.5 ' +
       '--runtime-version 1.12 ' + 
       '--job-dir=gs://holoselecta_objectdetection2/inception/{}/{}/ '.format(res,it) + 
       '--packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz,/tmp/pycocotools/pycocotools-2.0.tar.gz ' + 
       '--module-name object_detection.model_main ' + 
       '--region us-central1 ' + 
       '--config /content/gdrive/My\\ Drive/Masterarbeit/ObjectDetection/CloudConfigFiles/standard_p100_no_worker.yml ' + 
       '-- ' + 
       '--model_dir=gs://holoselecta_objectdetection2/inception/{}/{}/ '.format(res,it) + 
       '--pipeline_config_path=gs://holoselecta_objectdetection2/inception/{}/{}/pipeline.config'.format(res,it))
  print(cl_str)

960
cd /content/tf/research; gcloud ml-engine jobs submit training object_detection_`date +%m_%d_%Y_%H_%M_%S` --python-version 3.5 --runtime-version 1.12 --job-dir=gs://holoselecta_objectdetection2/inception/960/60/ --packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz,/tmp/pycocotools/pycocotools-2.0.tar.gz --module-name object_detection.model_main --region us-central1 --config /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/CloudConfigFiles/standard_p100_no_worker.yml -- --model_dir=gs://holoselecta_objectdetection2/inception/960/60/ --pipeline_config_path=gs://holoselecta_objectdetection2/inception/960/60/pipeline.config
960
cd /content/tf/research; gcloud ml-engine jobs submit training object_detection_`date +%m_%d_%Y_%H_%M_%S` --python-version 3.5 --runtime-version 1.12 --job-dir=gs://holoselecta_objectdetection2/inception/960/40/ --packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz,/tmp/pycocotools/pycocotools-2.0.tar.gz --module-name objec

In [0]:
!cd /content/tf/research; gcloud ml-engine jobs submit training object_detection_`date +%m_%d_%Y_%H_%M_%S` --python-version 3.5 --runtime-version 1.12 --job-dir=gs://holoselecta_objectdetection2/inception/960/10/ --packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz,/tmp/pycocotools/pycocotools-2.0.tar.gz --module-name object_detection.model_main --region us-central1 --config /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/CloudConfigFiles/standard_p100_no_worker.yml -- --model_dir=gs://holoselecta_objectdetection2/inception/960/10/ --pipeline_config_path=gs://holoselecta_objectdetection2/inception/960/10/pipeline.config

Job [object_detection_06_08_2019_12_31_40] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe object_detection_06_08_2019_12_31_40

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs object_detection_06_08_2019_12_31_40
jobId: object_detection_06_08_2019_12_31_40
state: QUEUED


In [0]:
!gcloud ai-platform jobs describe object_detection_05_19_2019_07_24_21

createTime: '2019-05-19T07:24:30Z'
endTime: '2019-05-19T07:35:56Z'
errorMessage: |-
  The replica master 0 exited with a non-zero status of 1. 
  Traceback (most recent call last):
    [...]
    File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/estimator/estimator.py", line 1075, in _get_features_and_labels_from_input_fn
      self._call_input_fn(input_fn, mode))
    File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/estimator/estimator.py", line 1162, in _call_input_fn
      return input_fn(**kwargs)
    File "/root/.local/lib/python3.5/site-packages/object_detection/inputs.py", line 525, in _train_input_fn
      batch_size=params['batch_size'] if params else train_config.batch_size)
    File "/root/.local/lib/python3.5/site-packages/object_detection/builders/dataset_builder.py", line 124, in build
      num_additional_channels=input_reader_config.num_additional_channels)
    File "/root/.local/lib/python3.5/site-packages/object_detection/data_decoders/tf_exam

In [0]:
!gsutil ls gs://holoselecta_objectdetection/resnet50/train/*

gs://holoselecta_objectdetection/resnet50/train/
gs://holoselecta_objectdetection/resnet50/train/checkpoint
gs://holoselecta_objectdetection/resnet50/train/events.out.tfevents.1558252346.cmle-training-11657413241799530257
gs://holoselecta_objectdetection/resnet50/train/graph.pbtxt
gs://holoselecta_objectdetection/resnet50/train/label_map.pbtxt
gs://holoselecta_objectdetection/resnet50/train/model.ckpt-1766.data-00000-of-00001
gs://holoselecta_objectdetection/resnet50/train/model.ckpt-1766.index
gs://holoselecta_objectdetection/resnet50/train/model.ckpt-1766.meta
gs://holoselecta_objectdetection/resnet50/train/model.ckpt-3468.data-00000-of-00001
gs://holoselecta_objectdetection/resnet50/train/model.ckpt-3468.index
gs://holoselecta_objectdetection/resnet50/train/model.ckpt-3468.meta
gs://holoselecta_objectdetection/resnet50/train/model.ckpt-5187.data-00000-of-00001
gs://holoselecta_objectdetection/resnet50/train/model.ckpt-5187.index
gs://holoselecta_objectdetection/resnet50/train/model.

In [0]:
import shutil
import os

!mkdir test_model
!gsutil -m cp -r gs://holoselecta_objectdetection/resnet50/train/* test_model

with open("test_model/README.md".format(timestamp), "+w") as f:
  f.write("# Model Description  \n")
  f.write("Model: SSD, Resnet 50, Focal Point Network (FPN), Shared Box Predictor, (aka RetinaNet)  \n")
  f.write("Input Resolution: 640 x 640  \n")
  f.write("Classes: 47  \n")
  f.write("Training Images: _Automat  \n")
  f.write("Dataset Timestamp: 2019_5_18_15_41_18  \n")
  f.write("Training Steps: 7000  \n")
  f.write("Final mAP@.50IOU = 0.86 ; mAP@.75IOU = 0.78  \n")
  f.write("Max Score Steps: 7000")

shutil.copy("model/frozen_inference_graph.pb", "test_model/frozen_inference_graph.pb")
shutil.copy("data/label_map.pbtxt", "test_model/label_map.pbtxt")
shutil.copy("pipeline.config", "test_model/pipeline.config")
shutil.copy("timestamp", "test_model/timestamp")

timestamp = "2019_5_19_17_37_00"
shutil.copytree("test_model", os.path.join(FLAGS_OD.model_dir, "Good", timestamp))

mkdir: cannot create directory ‘test_model’: File exists
Copying gs://holoselecta_objectdetection/resnet50/train/model.ckpt-1766.data-00000-of-00001...
Copying gs://holoselecta_objectdetection/resnet50/train/events.out.tfevents.1558252346.cmle-training-11657413241799530257...
Copying gs://holoselecta_objectdetection/resnet50/train/checkpoint...
Copying gs://holoselecta_objectdetection/resnet50/train/model.ckpt-1766.meta...
Copying gs://holoselecta_objectdetection/resnet50/train/model.ckpt-3468.meta...
Copying gs://holoselecta_objectdetection/resnet50/train/model.ckpt-1766.index...
Copying gs://holoselecta_objectdetection/resnet50/train/model.ckpt-3468.data-00000-of-00001...
Copying gs://holoselecta_objectdetection/resnet50/train/graph.pbtxt...
Copying gs://holoselecta_objectdetection/resnet50/train/model.ckpt-3468.index...
Copying gs://holoselecta_objectdetection/resnet50/train/label_map.pbtxt...
Copying gs://holoselecta_objectdetection/resnet50/train/model.ckpt-5187.data-00000-of-0000

FileExistsError: ignored

In [0]:
timestamp = "2019_5_19_17_39_00"
shutil.copytree("test_model", os.path.join(FLAGS_OD.model_dir, "Good", timestamp))

'/content/gdrive/My Drive/Masterarbeit/ObjectDetection/Models/Good/2019_5_19_17_39_00'

In [0]:
#save results
#clean bucket
if not os.path.exists("train"):
  os.mkdir("train")
else:
  for the_file in os.listdir("train"):
    file_path = os.path.join("train", the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
!gsutil -m cp -r gs://holoselecta_objectdetection/resent50/train/* train
now = datetime.now()
timestamp = "_".join([str(now.year), str(now.month), str(now.day), str(now.hour), str(now.minute), str(now.second)])
model_path = os.path.join(FLAGS_OD.model_dir,timestamp)
shutil.copytree("train", model_path)

Copying gs://holoselecta_objectdetection/train/checkpoint...
Copying gs://holoselecta_objectdetection/train/events.out.tfevents.1548195499.cmle-training-2435992349304765774...
Copying gs://holoselecta_objectdetection/train/model.ckpt-0.meta...
Copying gs://holoselecta_objectdetection/train/label_map.pbtxt...
Copying gs://holoselecta_objectdetection/train/model.ckpt-2000.data-00000-of-00001...
Copying gs://holoselecta_objectdetection/train/model.ckpt-0.data-00000-of-00001...
Copying gs://holoselecta_objectdetection/train/graph.pbtxt...
Copying gs://holoselecta_objectdetection/train/model.ckpt-2000.index...
Copying gs://holoselecta_objectdetection/train/model.ckpt-0.index...
Copying gs://holoselecta_objectdetection/train/model.ckpt-2000.meta...
Copying gs://holoselecta_objectdetection/train/pipeline.config...
Copying gs://holoselecta_objectdetection/train/timestamp...
Copying gs://holoselecta_objectdetection/train/eval_validation_data/events.out.tfevents.1548233381.cmle-training-69519515

In [0]:
!gsutil -m rm -r gs://holoselecta_objectdetection_2/train/*
!gsutil -m rm -r gs://holoselecta_objectdetection_2/data/*

# Tensorboard

In [0]:
class FLAGS_TB:
  # a temporary folder to run tensorboard on, doesn't seem to run on a Gdrive mounted folder
  tmp_graph_dir = "tmp_graph"
  
  # how many images fit horizontally in an image mosaic
  tb_mosaic_width = 5

In [0]:
import shutil
import os

if os.path.exists(FLAGS_TB.tmp_graph_dir):
  print("delete dir:")
  shutil.rmtree(FLAGS_TB.tmp_graph_dir)

delete dir:


In [0]:
import shutil

print("copy dir:")
shutil.copytree("gdrive/My Drive/Masterarbeit/ObjectDetection/Models/2019_1_23_9_13_50" ,FLAGS_TB.tmp_graph_dir)

copy dir:


'tmp_graph'

In [0]:
#!pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab

tbc = TensorBoardColab(graph_path=FLAGS_TB.tmp_graph_dir)

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://1d54274a.ngrok.io


# Test

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [0]:
# This is needed to display the images.
%matplotlib inline

In [0]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
model_timestamp = "2019_1_23_6_26_33"
PATH_TO_FROZEN_GRAPH = os.path.join(FLAGS_OD.model_dir,model_timestamp,"graph.pbtxt")

# List of the strings that is used to add correct label for each box.
label_map_timestamp = "2019_1_22_19_39_47"
PATH_TO_LABELS = os.path.join(FLAGS_OD.label_map_dir,"{}.pbtxt".format(label_map_timestamp))

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

/content/gdrive/My Drive/Masterarbeit/ObjectDetection/Models/2019_1_23_6_26_33/graph.pbtxt


DecodeError: ignored

In [0]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [0]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [0]:
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

# Create DataSet

In [0]:
!mkdir COCO

!wget http://images.cocodataset.org/zips/train2014.zip
!wget http://images.cocodataset.org/zips/val2014.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!wget http://images.cocodataset.org/zips/test2014.zip
!wget http://images.cocodataset.org/annotations/image_info_test2014.zip

--2019-05-21 19:09:42--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.97.227
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.97.227|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  50.7MB/s    in 4m 1s   

2019-05-21 19:13:43 (53.4 MB/s) - ‘train2014.zip’ saved [13510573713/13510573713]

--2019-05-21 19:13:44--  http://images.cocodataset.org/zips/val2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.233.27
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.233.27|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘val2014.zip’

val2014.zip         100%[===================>]   6.19G  53.3MB/s    in 2m 8s   

2019-05-21 19:15:52 (49.6 MB/s) - ‘

In [0]:
!unzip -q train2014.zip -d COCO
!unzip -q val2014.zip -d COCO
!unzip -q annotations_trainval2014.zip -d COCO
!unzip -q test2014.zip -d COCO
!unzip -q image_info_test2014.zip -d COCO

In [0]:
!rm train2014.zip
!rm val2014.zip
!rm annotations_trainval2014.zip
!rm test2014.zip
!rm image_info_test2014.zip

In [0]:
!du -sh COCO/test2014
!du -sh COCO/train2014
!du -sh COCO/val2014

6.3G	COCO/test2014
13G	COCO/train2014
6.3G	COCO/val2014


In [0]:
import json
import pprint

def present_keys_recursive(var):
  if isinstance(var, dict):
    d = dict()
    for k,v in var.items():
      d[k] = present_keys_recursive(v)
  elif isinstance(var, list):
    d = [present_keys_recursive(var[0])]
  else:
    d = type(var)
  return d

with open("COCO/annotations/image_info_test2014.json", "r") as file:
  j = json.load(file)

layout = present_keys_recursive(j)

pp = pprint.PrettyPrinter(indent=2)

pp.pprint(layout)

{ 'categories': [ { 'id': <class 'int'>,
                    'name': <class 'str'>,
                    'supercategory': <class 'str'>}],
  'images': [ { 'coco_url': <class 'str'>,
                'date_captured': <class 'str'>,
                'file_name': <class 'str'>,
                'height': <class 'int'>,
                'id': <class 'int'>,
                'license': <class 'int'>,
                'width': <class 'int'>}],
  'info': { 'contributor': <class 'str'>,
            'date_created': <class 'str'>,
            'description': <class 'str'>,
            'url': <class 'str'>,
            'version': <class 'str'>,
            'year': <class 'int'>},
  'licenses': [ { 'id': <class 'int'>,
                  'name': <class 'str'>,
                  'url': <class 'str'>}]}


In [0]:
!cd /content/tf/research; python object_detection/dataset_tools/create_coco_tf_record.py --logtostderr --train_image_dir="/content/COCO/train2014" --val_image_dir="/content/COCO/val2014" --train_annotations_file="/content/COCO/annotations/image_info_test2014.json.json" --val_annotations_file="/content/COCO/annotations/instances_val2014.json" --output_dir="/content/COCO"

Traceback (most recent call last):
  File "object_detection/dataset_tools/create_coco_tf_record.py", line 45, in <module>
    from object_detection.dataset_tools import tf_record_creation_util
ModuleNotFoundError: No module named 'object_detection'


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import io
import json
import os
import contextlib2
import numpy as np
import PIL.Image

from pycocotools import mask
import tensorflow as tf

from object_detection.dataset_tools import tf_record_creation_util
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

In [0]:
tf.flags.DEFINE_boolean('include_masks', False,
                        'Whether to include instance segmentations masks '
                        '(PNG encoded) in the result. default: False.')
tf.flags.DEFINE_string('train_image_dir', '/content/COCO/train2014',
                       'Training image directory.')
tf.flags.DEFINE_string('val_image_dir', '/content/COCO/val2014',
                       'Validation image directory.')
tf.flags.DEFINE_string('train_annotations_file', '/content/COCO/annotations/instances_train2014.json',
                       'Training annotations JSON file.')
tf.flags.DEFINE_string('val_annotations_file', '/content/COCO/annotations/instances_val2014.json',
                       'Validation annotations JSON file.')
tf.flags.DEFINE_string('output_dir', '/content/COCO/', 'Output data directory.')

In [0]:
def create_tf_example(image,
                      annotations_list,
                      image_dir,
                      category_index,
                      include_masks=False):
  """Converts image and annotations to a tf.Example proto.
  Args:
    image: dict with keys:
      [u'license', u'file_name', u'coco_url', u'height', u'width',
      u'date_captured', u'flickr_url', u'id']
    annotations_list:
      list of dicts with keys:
      [u'segmentation', u'area', u'iscrowd', u'image_id',
      u'bbox', u'category_id', u'id']
      Notice that bounding box coordinates in the official COCO dataset are
      given as [x, y, width, height] tuples using absolute coordinates where
      x, y represent the top-left (0-indexed) corner.  This function converts
      to the format expected by the Tensorflow Object Detection API (which is
      which is [ymin, xmin, ymax, xmax] with coordinates normalized relative
      to image size).
    image_dir: directory containing the image files.
    category_index: a dict containing COCO category information keyed
      by the 'id' field of each category.  See the
      label_map_util.create_category_index function.
    include_masks: Whether to include instance segmentations masks
      (PNG encoded) in the result. default: False.
  Returns:
    example: The converted tf.Example
    num_annotations_skipped: Number of (invalid) annotations that were ignored.
  Raises:
    ValueError: if the image pointed to by data['filename'] is not a valid JPEG
  """
  FLAGS = tf.app.flags.FLAGS
  image_height = image['height']
  image_width = image['width']
  filename = image['file_name']
  image_id = image['id']

  full_path = os.path.join(image_dir, filename)
  with tf.gfile.GFile(full_path, 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = PIL.Image.open(encoded_jpg_io)
  key = hashlib.sha256(encoded_jpg).hexdigest()

  xmin = []
  xmax = []
  ymin = []
  ymax = []
  is_crowd = []
  category_names = []
  category_ids = []
  area = []
  encoded_mask_png = []
  num_annotations_skipped = 0
  for object_annotations in annotations_list:
    (x, y, width, height) = tuple(object_annotations['bbox'])
    if width <= 0 or height <= 0:
      num_annotations_skipped += 1
      continue
    if x + width > image_width or y + height > image_height:
      num_annotations_skipped += 1
      continue
    xmin.append(float(x) / image_width)
    xmax.append(float(x + width) / image_width)
    ymin.append(float(y) / image_height)
    ymax.append(float(y + height) / image_height)
    is_crowd.append(object_annotations['iscrowd'])
    category_id = int(object_annotations['category_id'])
    category_ids.append(category_id)
    category_names.append(category_index[category_id]['name'].encode('utf8'))
    area.append(object_annotations['area'])

    if include_masks:
      run_len_encoding = mask.frPyObjects(object_annotations['segmentation'],
                                          image_height, image_width)
      binary_mask = mask.decode(run_len_encoding)
      if not object_annotations['iscrowd']:
        binary_mask = np.amax(binary_mask, axis=2)
      pil_image = PIL.Image.fromarray(binary_mask)
      output_io = io.BytesIO()
      pil_image.save(output_io, format='PNG')
      encoded_mask_png.append(output_io.getvalue())
  feature_dict = {
      'image/height':
          dataset_util.int64_feature(image_height),
      'image/width':
          dataset_util.int64_feature(image_width),
      'image/filename':
          dataset_util.bytes_feature(filename.encode('utf8')),
      'image/source_id':
          dataset_util.bytes_feature(str(image_id).encode('utf8')),
      'image/key/sha256':
          dataset_util.bytes_feature(key.encode('utf8')),
      'image/encoded':
          dataset_util.bytes_feature(encoded_jpg),
      'image/format':
          dataset_util.bytes_feature('jpeg'.encode('utf8')),
      'image/object/bbox/xmin':
          dataset_util.float_list_feature(xmin),
      'image/object/bbox/xmax':
          dataset_util.float_list_feature(xmax),
      'image/object/bbox/ymin':
          dataset_util.float_list_feature(ymin),
      'image/object/bbox/ymax':
          dataset_util.float_list_feature(ymax),
      'image/object/class/text':
          dataset_util.bytes_list_feature(category_names),
      'image/object/is_crowd':
          dataset_util.int64_list_feature(is_crowd),
      'image/object/area':
          dataset_util.float_list_feature(area),
  }
  if include_masks:
    feature_dict['image/object/mask'] = (
        dataset_util.bytes_list_feature(encoded_mask_png))
  example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
  return key, example, num_annotations_skipped

In [0]:
def _create_tf_record_from_coco_annotations(
    annotations_file, image_dir, output_path, include_masks, num_shards):
  """Loads COCO annotation json files and converts to tf.Record format.
  Args:
    annotations_file: JSON file containing bounding box annotations.
    image_dir: Directory containing the image files.
    output_path: Path to output tf.Record file.
    include_masks: Whether to include instance segmentations masks
      (PNG encoded) in the result. default: False.
    num_shards: number of output file shards.
  """
  FLAGS = tf.app.flags.FLAGS
  with contextlib2.ExitStack() as tf_record_close_stack, \
      tf.gfile.GFile(annotations_file, 'r') as fid:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
        tf_record_close_stack, output_path, num_shards)
    groundtruth_data = json.load(fid)
    images = groundtruth_data['images']
    category_index = label_map_util.create_category_index(
        groundtruth_data['categories'])

    annotations_index = {}
    if 'annotations' in groundtruth_data:
      tf.logging.info(
          'Found groundtruth annotations. Building annotations index.')
      for annotation in groundtruth_data['annotations']:
        image_id = annotation['image_id']
        if image_id not in annotations_index:
          annotations_index[image_id] = []
        annotations_index[image_id].append(annotation)
    missing_annotation_count = 0
    for image in images:
      image_id = image['id']
      if image_id not in annotations_index:
        missing_annotation_count += 1
        annotations_index[image_id] = []
    tf.logging.info('%d images are missing annotations.',
                    missing_annotation_count)

    total_num_annotations_skipped = 0
    for idx, image in enumerate(images):
      if idx % 100 == 0:
        tf.logging.info('On image %d of %d', idx, len(images))
      annotations_list = annotations_index[image['id']]
      _, tf_example, num_annotations_skipped = create_tf_example(
          image, annotations_list, image_dir, category_index, include_masks)
      total_num_annotations_skipped += num_annotations_skipped
      shard_idx = idx % num_shards
      output_tfrecords[shard_idx].write(tf_example.SerializeToString())
    tf.logging.info('Finished writing, skipped %d annotations.',
                    total_num_annotations_skipped)

In [0]:
def main(_):
  FLAGS = tf.app.flags.FLAGS
  
  assert FLAGS.train_image_dir, '`train_image_dir` missing.'
  assert FLAGS.val_image_dir, '`val_image_dir` missing.'
  assert FLAGS.train_annotations_file, '`train_annotations_file` missing.'
  assert FLAGS.val_annotations_file, '`val_annotations_file` missing.'

  if not tf.gfile.IsDirectory(FLAGS.output_dir):
    tf.gfile.MakeDirs(FLAGS.output_dir)
  train_output_path = os.path.join(FLAGS.output_dir, 'coco_train.record')
  val_output_path = os.path.join(FLAGS.output_dir, 'coco_val.record')
  testdev_output_path = os.path.join(FLAGS.output_dir, 'coco_testdev.record')

  _create_tf_record_from_coco_annotations(
      FLAGS.train_annotations_file,
      FLAGS.train_image_dir,
      train_output_path,
      FLAGS.include_masks,
      num_shards=100)
  _create_tf_record_from_coco_annotations(
      FLAGS.val_annotations_file,
      FLAGS.val_image_dir,
      val_output_path,
      FLAGS.include_masks,
      num_shards=10)

In [0]:
tf.app.run(main)

INFO:tensorflow:Found groundtruth annotations. Building annotations index.


I0521 20:08:42.455298 140550975006592 <ipython-input-9-8f22e8fede56>:24] Found groundtruth annotations. Building annotations index.


INFO:tensorflow:702 images are missing annotations.


I0521 20:08:43.155018 140550975006592 <ipython-input-9-8f22e8fede56>:37] 702 images are missing annotations.


INFO:tensorflow:On image 0 of 82783


...

I0521 20:20:13.739274 140550975006592 <ipython-input-9-8f22e8fede56>:42] On image 40500 of 40504


INFO:tensorflow:Finished writing, skipped 1 annotations.


I0521 20:20:13.784533 140550975006592 <ipython-input-9-8f22e8fede56>:50] Finished writing, skipped 1 annotations.


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
!rm -r COCO/test2014
!rm -r COCO/train2014 
!rm -r COCO/val2014
!rm -r COCO/annotations

In [0]:
!cp COCO/coco_train.record* /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/Datasets/COCO/train
!cp COCO/coco_val.record* /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/Datasets/COCO/val